#### Read in the data

## Notes, resources, other...

In [ ]:
'''
https://stackoverflow.com/questions/22219004/grouping-rows-in-list-in-pandas-groupby

# below creates unique set of keys comprised of first 4 columns (C/A, STATION, UNIT, etc.)
new_set = set(zip(df['C/A'], df['UNIT'], df['SCP'], df['STATION']))

# below works as an initial groupby consolidation
df.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE']).count()
'''

In [26]:
import pandas as pd
txt_docs_list = ['turnstile_160409.txt', 'turnstile_180310.txt', 'turnstile_180317.txt',
                 'turnstile_180324.txt', 'turnstile_180331.txt', 'turnstile_180407.txt']

df_list = []
for each in range(1):
    df_list.append(pd.read_csv(txt_docs_list[each]))
    
df = pd.concat(df_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194516 entries, 0 to 194515
Data columns (total 11 columns):
C/A                                                                     194516 non-null object
UNIT                                                                    194516 non-null object
SCP                                                                     194516 non-null object
STATION                                                                 194516 non-null object
LINENAME                                                                194516 non-null object
DIVISION                                                                194516 non-null object
DATE                                                                    194516 non-null object
TIME                                                                    194516 non-null object
DESC                                                                    194516 non-null object
ENTRIES                           

# Challenge 1. Build the dictionary

In [34]:
df['DATETIME'] = df['DATE'] + ' ' + df['TIME']
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194516 entries, 0 to 194515
Data columns (total 12 columns):
C/A                                                                     194516 non-null object
UNIT                                                                    194516 non-null object
SCP                                                                     194516 non-null object
STATION                                                                 194516 non-null object
LINENAME                                                                194516 non-null object
DIVISION                                                                194516 non-null object
DATE                                                                    194516 non-null object
TIME                                                                    194516 non-null object
DESC                                                                    194516 non-null object
ENTRIES                           

#### concat 4 ids for new column to avoid groupby, then sort by dates, then build list for unique vals in each date...

In [61]:
df['turnstile'] = df['C/A'] + '_' + df['UNIT'] + '_' + df['SCP'] + '_' + df['STATION']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194516 entries, 0 to 194515
Data columns (total 13 columns):
C/A          194516 non-null object
UNIT         194516 non-null object
SCP          194516 non-null object
STATION      194516 non-null object
LINENAME     194516 non-null object
DIVISION     194516 non-null object
DATE         194516 non-null object
TIME         194516 non-null object
DESC         194516 non-null object
ENTRIES      194516 non-null int64
EXITS        194516 non-null int64
DATETIME     194516 non-null datetime64[ns]
turnstile    194516 non-null object
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 19.3+ MB


In [62]:
df.columns = df.columns.str.strip()
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS', 'DATETIME', 'turnstile'],
      dtype='object')

In [74]:
df.groupby(['turnstile', 'DATE'])['ENTRIES'].apply(list)
df2

turnstile                          DATE      
A002_R051_02-00-00_59 ST           04/02/2016    [5600838, 5600863, 5600877, 5600922, 5601172, ...
                                   04/03/2016    [5601621, 5601638, 5601643, 5601722, 5601917, ...
                                   04/04/2016    [5602244, 5602256, 5602297, 5602326, 5602456, ...
                                   04/05/2016        [5603671, 5603729, 5603886, 5604169, 5604944]
                                   04/06/2016    [5605129, 5605137, 5605168, 5605278, 5605583, ...
                                   04/07/2016    [5606603, 5606619, 5606656, 5606803, 5607108, ...
                                   04/08/2016    [5608161, 5608184, 5608228, 5608402, 5608800, ...
A002_R051_02-00-01_59 ST           04/02/2016    [5148548, 5148569, 5148583, 5148636, 5148817, ...
                                   04/03/2016    [5149186, 5149205, 5149209, 5149260, 5149424, ...
                                   04/04/2016    [5149693, 5149

In [97]:
# now figure out how to turn move the series into df column..
## separate df that returns only same # of rows as this series (basically agg'd by turnstile & DATE) (32,125 rows)
## OR plug back into originally "grouby'd" df

import numpy as np
entries_list = list(df.groupby(['turnstile', 'DATE'])['ENTRIES'].apply(list))
exits_list = list(df.groupby(['turnstile', 'DATE'])['EXITS'].apply(list))
'''
df3 = df.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE']).count() #df2 already exists above
df3['entries_list'] = list(entries_list_srs)
df3['entries_dcount'] = (df[entries_list][-1] - df[entries_list][0])
df3['exits_list'] = list(exits_list_srs)
df3['exits_dcount'] = (df[exits_list][-1] - df[exits_list][0])

df3.head()
'''

entries_count = []
for e in entries_list:
    entries_count.append(e[-1] - e[0])

entries_count[:30000:300]

[646,
 1418,
 1487,
 2681,
 906,
 372,
 257,
 14,
 1924,
 1055,
 1448,
 1348,
 1576,
 336,
 670,
 2401,
 1820,
 2451,
 917,
 1789,
 389,
 249,
 1178,
 2,
 482,
 35,
 3577,
 999,
 1816,
 474,
 1339,
 1629,
 3461,
 1630,
 0,
 3136,
 781,
 192,
 699,
 1026,
 593,
 0,
 1591,
 782,
 2713,
 1378,
 565,
 68,
 408,
 871,
 14,
 2162,
 1250,
 263,
 1218,
 1095,
 0,
 231,
 576,
 7500,
 1428,
 578,
 0,
 1032,
 277,
 389,
 696,
 725,
 330,
 125,
 0,
 3,
 1473,
 2010,
 1032,
 1412,
 68,
 283,
 1862,
 2741,
 1960,
 2932,
 481,
 729,
 386,
 1161,
 -4666,
 2300,
 245,
 428,
 2191,
 1463,
 1099,
 0,
 487,
 0,
 694,
 2243,
 1600,
 0]

#### Now count for each day...